### 허깅페이스 Lanchain

#### fill-mask 등 현재 langchain과 huggingface 연동이 지원되지 않을 때 직접 pipeline 구축하여 사용

In [12]:
import torch, os
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline

# Model and tokenizer
model_name = 'bigscience/bloomz-7b1'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

# Question answering pipeline
qa_pipeline = pipeline('question-answering', model=model, tokenizer=tokenizer)

# Question and context
question = "Who is Son Heung Min?"
context = """
    Son Heung-min is a South Korean professional footballer who plays as a forward for Premier League club Tottenham Hotspur and captains the South Korea national team. 
    Considered one of the best forwards in the world, he is known for his explosive speed, finishing, and two-footed ability.
"""

# Answering the question
result = qa_pipeline(question=question, context=context)
print(f"Answer: {result['answer']}")

c:\Users\whdgh\Desktop\projects\Langchain_Embedding\.venv\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\whdgh\.cache\huggingface\hub\models--bigscience--bloomz-7b1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of BloomForQuestionAnswering were not initialized from the model checkpo

Answer:  for Premier League club Tottenham Hotspur and captains


#### huggingface와 langchain pipeline 지원될 때 아래와 같이 연동하여 사용

In [13]:
import configparser
config = configparser.ConfigParser()
config.read('config.ini')

import os
os.environ['HUGGINGFACEHUB_API_TOKEN'] = config['HUGGINGFACE_API']['HUGGINGFACEHUB_API_TOKEN']

from langchain import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFaceHub

# HuggingFace Repository ID
repo_id = 'mistralai/Mistral-7B-Instruct-v0.3'
# repo_id = 'bigscience/bloomz-7b1'

# 질의내용
question = "Who is Son Heung Min?"

# 템플릿
template = """Question: {question}

Answer: """

# 프롬프트 템플릿 생성
prompt = PromptTemplate(template=template, input_variables=["question"])

# HuggingFaceHub 객체 생성
llm = HuggingFaceHub(
    repo_id=repo_id, 
    model_kwargs={"temperature": 0.2, 
                  "max_length": 128}
)

# LLM Chain 객체 생성
llm_chain = LLMChain(prompt=prompt, llm=llm)

# 실행
print(llm_chain.run(question=question))


Question: Who is Son Heung Min?

Answer:  Son Heung-min is a South Korean professional footballer who plays as a winger for English Premier League club Tottenham Hotspur and the South Korea national team. He is known for his pace, dribbling skills, and ability to score goals. Son joined Tottenham from Bayer Leverkusen in 2015 and has since become a key player for the club, helping them to reach the Champions League final in 2019. He
